In [1]:
import math
import copy
import numpy as np

In [4]:
class Node():
    
    def __init__(self, size, costs, sortedEdges, allSortedEdges, parent_constr, extra_constr=None):
    
        self.size = size # Number of cities
        self.costs = costs # Distance matrix
        self.sortedEdges = sortedEdges
        self.allSortedEdges = allSortedEdges
        self.extra_constr = extra_constr
        self.constraints = self.determine_constr(parent_constr)
        self.lowerBound = self.computeLowerBound()
        
    #This method calculates the simple lower bound (SB)
    #This is the sum over all vertices of the two smallest allowed edges
    def computeLowerBound(self):
        lb = 0
        for i in range(self.size):
            lower = 0
            t = 0
            for j in range(self.size):
                if self.constraints [i][j] == 1:
                    lower += self.costs[i][j]
                    t += 1
        tt = 0
        while t < 2:
            shortest = self.sortedEdges[i][tt]
            if self.constraints[i][shortest] == 2:
                lower += self.costs[i][shortest]
                t += 1
            tt += 1
            if tt >= self.size:
                lower = math.inf
                break
        lb += lower
        return lb

    #This method determines the constraints of a node 
    #based on the parent constaints and the extra constraint
    def determine_constr(self, parent_constr):
        constraints = copy.deepcopy(parent_constr)
        if self.extra_constr == None:
            return constraints
        fr = self.extra_constr[0]
        to = self.extra_constr[1]
        constraints[fr][to] = self.extra_constr[2]
        constraints[to][fr] = self.extra_constr[2]
        for i in range(2):
            constraints = self.removeEdges(constraints)
            constraints = self.addEdges(constraints)
        
        return constraints

    #50
    #This method exclude edges when:
    #1. Two other edges adjacent to the same vertex are included
    #2. Including the edge would create a subtour
    def removeEdges(self, constraints):
        for i in range(self.size):
            t = 0
        for j in range(self.size):
            if (i != j) and (constraints[i][j] == 1):
                t += 1
        if t >= 2:
            for j in range(self.size):
                if constraints[i][j] == 2:
                    constraints[i][j] = 0
                    constraints[j][i] = 0
        for i in range(self.size):
            for j in range(self.size):
                t = 1
                prev = i
                fr = j
                cycle = False
                nextOne = self.next_one(prev, fr, constraints)
                while (nextOne[0]):
                    t += 1
                    next = nextOne[1]
                    if next == i:
                        cycle = True
                        break
                    if t > self.size:
                        break
                    prev = fr
                    fr = next
                    nextOne = self.next_one(prev, fr, constraints)
                if (cycle) and (t < self.size) and (constraints[i][j] == 2):
                    constraints[i][j] = 0
                    constraints[j][i] = 0
        return constraints
    
    #This method checks if all but two edges adjacent to a vertex are excluded
    #If so, these edges are included
    def addEdges(self, constraints):
        for i in range(self.size):
            t = 0
            for j in range(self.size):
                if constraints[i][j] == 0:
                    t += 1
            if t == self.size - 2:
                for j in range(self.size):
                    if constraints[i][j] == 2:
                        constraints[i][j] = 1
                        constraints[j][i] = 1
        return constraints
    
    #Determines whether there exists an included edge that starts in fs  and does not end in prev
    #If so, it also returns the endpoint of this edge.
    def next_one (self, prev, fr, constraints):
        for j in range(self.size):
            if (constraints[fr][j] == 1) and (j != prev):
                return [True ,j]
        return [False]
    
    #Determines if a node representd a full tour by checking whether from every vertex
    #there are exactly 2 included edges and all other edges are excluded
    def isTour(self):
        for i in range(self.size):
            num_zero = 0
            num_one = 0
            for j in range(self.size):
                if self.constraints[i][j] == 0:
                    num_zero += 1
                elif self.constraints[i][j] == 1:
                    num_one += 1
        if (num_zero != self.size - 2) or (num_one != 2):
            return False
        return True

    #Checks if a node contains a subtour
    def contains_subtour(self):
        for i in range(self.size):
            next = self.next_one(i, i, self.constraints)
            if next[0]:
                prev = i
                fr = next[1]
                t = 1
                next = self.next_one(prev, fr, self.constraints)
            while next[0]:
                t += 1
                prev = fr
                fr = next[1]
                if (fr == i) and (t < self.size):
                    return True
                next = self.next_one(prev, fr, self.constraints)
                if t == self.size:
                    return False
        return False
    
    #Assumes the node represents a tour and returns the length
    def tourLength(self):
        length = 0
        fr = 0
        to = self.next_one(fr, fr, self.constraints)[1]
        for i in range(self.size):
            length += self.costs[fr][to]
            temp = fr
            fr = to
            to = self.next_one(temp, to, self.constraints)[1]
        return length

    #Determines the next constraint according to the branching strategy
    #lexicographic order (LG)
    def next_constraint(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.constraints[i][j] == 2:
                    return [i,j]
                
    #If a node represents a tour
    #returns a string with the order of the vertices in the tour
    def __str__(self):
        if self.isTour():
            result = "0"
            fr = 0
            to = None
            for j in range(self.size):
                if self.constraints[fr][j] == 1:
                    to = j
                    result += "−" + str(j)
                    break
            for i in range(self.size - 1):
                for j in range(self.size):
                    if (self.constraints[to][j] == 1) and (j != fr):
                        result += "−" + str(j)
                        fr = to
                        to = j
                        break
            return result
        else:
            print("This node is not a tour")

In [3]:
#from Node import Node
import math
import time
import copy

class TSP():
    
    def __init__(self, size, costs, bestTour=math.inf):
        self.size = size
        self.costs = costs
        self.bestTour = bestTour
        self.bestNode = None
        self.bestNodeTime = 0
        self.num_createdNodes = 0
        self.num_prunedNodes = 0
        self.sortedEdges = self.sort_edges()
        self.allSortedEdges = self.sort_allEdges()
        
    def findSolution(self):
        root = self.create_root()
        self.num_createdNodes += 1
        T1 = time.perf_counter()
        self.BranchAndBound(root)
        T2 = time.perf_counter()
        print("−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−")
        print("The shortest tour is:", self.bestNode)
        print("It has a length of", self.bestTour, "km")
        print("Found in", T2-T1, "seconds")
        print("Best tour was found after:", self.bestNodeTime, "seconds")
        print("Number of nodes created: ", self.num_createdNodes)
        print("Number of nodes pruned: ", self. num_prunedNodes)
        
    #sorts the edges of the distance matrix per row returns matirix where each
    #row i contains the numbers 0 <= k <= (self.size-1) in increasing costs of the edges (i,k)
    def sort_edges(self):
        result = []
        for i in range(self.size):
            result.append ([x for (y, x) in sorted(zip(self.costs[i], list(range(self.size))))])
        return result
    
    #sorts all edges of distance matrix
    #returns list of pairs [i,j] in order of increasing costs
    def sort_allEdges(self):
        edges = []
        lengths = []
        for i in range(self.size):
            for j in range(i+1, self.size):
                edges.append ([i, j])
                lengths.append(c[i][j])
        result = [z for (l, z) in sorted(zip(lengths, edges))]
        return result
    
    def create_root(self):
        no_constraints = []
        for i in range(self.size):
            row_i = []
            for j in range(self.size):
                if (i != j):
                    row_i.append(2)
                else:
                    row_i.append (0)
            no_constraints.append(row_i)
        root = Node(self.size, self.costs, self.sortedEdges, self.allSortedEdges, no_constraints)
        return root
    
    def BranchAndBound(self, node):
        if node.isTour():
            if node.tourLength() < self.bestTour:
                self.bestTour = node.tourLength()
                self.bestNode = node
                self.bestNodeTime = time.perf_counter()
                print("Found better tour: ", self.bestNode, "of length", self.bestTour, "km")
        else:
            new_constraint = copy.copy(node.next_constraint())
            new_constraint.append(1)
            leftChild = Node(self.size, self.costs, self.sortedEdges, self.allSortedEdges, node.constraints, new_constraint)
            new_constraint[2] = 0
            rightChild = Node(self.size, self.costs, self.sortedEdges, self.allSortedEdges, node.constraints, new_constraint)
            self.num_createdNodes += 2
            
            if self.num_createdNodes%401 == 0:
                print("Number of nodes created so far: ", self.num_createdNodes)
                print("Number of nodes pruned so far: ", self.num_prunedNodes)
            if self.num_createdNodes%51 == 0:
                print(".")
            if (leftChild.contains_subtour()) or (leftChild.lowerBound > 2 * self.bestTour):
                leftChild = None
                self.num_prunedNodes += 1
            if (rightChild.contains_subtour()) or (rightChild.lowerBound > 2 * self.bestTour):
                rightChild = None
                self.num_prunedNodes += 1
            if (leftChild != None) and (rightChild == None):
                self.BranchAndBound(leftChild)
            elif (leftChild == None) and (rightChild != None):
                self. BranchAndBound(rightChild)
            elif (leftChild != None) and (rightChild != None):
                if leftChild.lowerBound <= rightChild.lowerBound:
                    if leftChild.lowerBound < 2 * self.bestTour:
                        self.BranchAndBound(leftChild)
                    else:
                        leftChild = None
                        self.num_prunedNodes += 1
                    if rightChild.lowerBound < 2*self.bestTour:
                        self.BranchAndBound(rightChild)
                    else:
                        rightChild = None
                        self.num_prunedNodes += 1
                else:
                    if rightChild.lowerBound < 2*self.bestTour:
                        self.BranchAndBound(rightChild)
                    else:
                        rightChild = None
                        self.num_prunedNodes += 1
                    if leftChild.lowerBound < 2*self.bestTour:
                        self.BranchAndBound(leftChild)
                    else:
                        leftChild = None
                        self.num_prunedNodes += 1
       
    #Determines the next constraint using IL
    def next_constraint(self):
        for edge in self.allSortedEdges:
            i = edge[0]
            j = edge[1]
            if self.constraints[i][j] == 2:
                return edge
            
    #Calculates lower bound OT
    def computeLowerBound2 (self):
        lb = 0
        onetree = np.zeros((self.size, self.size), np.int8)
        t = 0
        for i in range(1, self.size):
            for j in range(i + 1, self.size):
                if self.constraints[i][j] == 1:
                    onetree[i][j] = 1
                    onetree[j][i] = 1
                    t += 1
                    lb += self.costs[i][j]
        for edge in self.allSortedEdges:
            if t >= self.size - 1:
                break
            i = edge[0]
            j = edge[1]
            if (self.constraints[i][j] == 2) and (i != 0):
                onetree[i][j] = 1
                onetree[j][i] = 1
            if self.onetree_contains_cycle(onetree):
                onetree[i][j] = 0
                onetree[j][i] = 0
            else:
                t += 1
                lb += self.costs[i][j]
        t = 0
        for j in range(self.size):
            if self.constraints[0][j] == 1:
                onetree[0][j] = 1
                onetree[j][0] = 1
                lb += self.costs[0][j]
                t += 1
        tt = 0
        while t < 2:
            shortest = self.sortedEdges[0][ tt]
            if self.constraints[0][shortest] == 2:
                onetree[0][shortest] = 1
                onetree[shortest][0] = 1
                lb += self.costs[0][shortest]
                t += 1
        tt += 1
        return lb

In [5]:
#Brute force algorithm
import itertools
import math

minLength = math.inf
minTour = []

for tour in itertools.permutations(list(range(1, n))):
    fr = 0
    length = 0
    count = 0
    while count < n - 1:
        to = tour[count]
        length += distances[fr][to]
        fr = to
        count += 1
    length += distances[fr][0]
    if length < minLength:
        minLength = length
        minTour = tour
minTour = (0,) + minTour + (0,)
print("Shortest tour is: ", minTour)
print("It has a length of: ", minLength , "km")

NameError: name 'n' is not defined